In [ ]:
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import Ollama  # Solution locale
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import warnings
warnings.filterwarnings('ignore')
import tqdm as notebook_tqdm

c:\Users\Axel\anaconda3\envs\mini_rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 1. Charger vos documents
loader = PyPDFLoader("TP8_cnn.pdf")  # ou TextLoader, DirectoryLoader
documents = loader.load()

# 2. Découper en chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # taille des chunks
    chunk_overlap=200,    # chevauchement pour garder le contexte
    length_function=len
)
chunks = text_splitter.split_documents(documents)

print(f"Nombre de chunks créés : {len(chunks)}")

Nombre de chunks créés : 13


In [32]:
# 1. Choisir le modèle d'embedding
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'}  # 'cuda' si GPU
)

# 2. Créer la base vectorielle FAISS
vector_store = FAISS.from_documents(chunks, embedding_model)

# 3. Sauvegarder la base
vector_store.save_local("faiss_index")
#vector_store = FAISS.load_local("faiss_index", embedding_model,allow_dangerous_deserialization=True)

#print("Base FAISS créée et sauvegardée !")

In [33]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

In [34]:
llm = Ollama(
    model="mistral",
    temperature=0.1,  # Pour des réponses plus factuelles
    num_predict=512   # Limite la longueur des réponses
)

In [35]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True
)

In [36]:
question = "Quelle est la motivation du TP8 Deep Learning et Réseau de Neurones Convolutionnel ?"
response = qa_chain({"query" : question}, return_only_outputs=False)
print("Réponse :", response["result"])
print("Source :", response["source_documents"][0].page_content)

Réponse :  The motivation for TP8 (Deep Learning and Convolutional Neural Networks) is to implement a variant of neural networks that are particularly useful for image processing. This variant, known as CNN (Convolutional Neural Network), will teach students how to implement it, use it, and analyze it, but they will be restricted to an Arduino DUE which has less computing power compared to computers.
Source : TP8 : Deep Learning et Réseau de Neurones
Convolutionnel
Axel Bröns
axel.brons@edu.ece.fr
Valentin Kocijancic
valentin.kocijancic@edu.ece.fr
Lyon, le 30 avril 2025
Nous attestons que ce travail est original, qu’il est le fruit d’un travail commun au binôme et
qu’il a été rédigé de manière autonome.


In [23]:
question = "Summarize me the TP8 on convolutional network and deep learning."
response = qa_chain(question)
print(response['result'])

AssertionError: 

In [14]:
print(response['result'])

 The TP8 is about a project involving Convolutional Neural Networks (CNN) in Deep Learning, which was completed by Axel Bröns and Valentin Kocijancic on April 30, 2025. The goal of the project was to implement a variant of neural networks that are particularly useful for image processing, known as CNNs.

The team explained the concept of convolution in their work, stating that it helps bring out certain features of images and extract important characteristics from them to optimize the neural network. They also discussed padding during convolutions to maintain the same output matrix size. Maxpooling was used to reduce the image size while minimizing loss of important information and noise reduction.

The team provided an algorithm for their CNN implementation, and they mentioned that their kernel is symmetric, so it doesn't need to be inverted. The motivation behind this project was to learn how to implement, use, and analyze CNNs on a limited-power Arduino DUE compared to computers.

T